In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Kidney_Papillary_Cell_Carcinoma"
cohort = "GSE40912"

# Input paths
in_trait_dir = "../../input/GEO/Kidney_Papillary_Cell_Carcinoma"
in_cohort_dir = "../../input/GEO/Kidney_Papillary_Cell_Carcinoma/GSE40912"

# Output paths
out_data_file = "../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/GSE40912.csv"
out_gene_data_file = "../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/gene_data/GSE40912.csv"
out_clinical_data_file = "../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/clinical_data/GSE40912.csv"
json_path = "../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Gene Expression Data Availability
# Based on the series title, summary, and design, this dataset contains gene expression data
is_gene_available = True  

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Kidney Papillary Cell Carcinoma):
# From the background information, we can see this is a clear cell RCC study
# Row 14 contains patient status info that can be used as our trait
trait_row = 14  

# For age:
# Row 4 contains age information
age_row = 4  

# For gender:
# Row 3 contains gender information
gender_row = 3  

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert patient status to binary: 1 for cancer death, 0 for alive without cancer."""
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip().lower()
        if 'cancer-specific death' in value:
            return 1
        elif 'alive without cancer' in value:
            return 0
    return None

def convert_age(value):
    """Convert age to continuous numeric value."""
    if isinstance(value, str) and ':' in value:
        try:
            # Extract the part after colon and convert to integer
            age_str = value.split(':', 1)[1].strip()
            return int(age_str)
        except (ValueError, IndexError):
            pass
    return None

def convert_gender(value):
    """Convert gender to binary: 0 for female, 1 for male."""
    if isinstance(value, str) and ':' in value:
        gender = value.split(':', 1)[1].strip().lower()
        if 'female' in gender:
            return 0
        elif 'male' in gender:
            return 1
    return None

# 3. Save Metadata
# Determine if trait data is available (trait_row is not None)
is_trait_available = trait_row is not None

# Initial filtering on usability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Check if trait data is available
if trait_row is not None:
    # Create a proper clinical data DataFrame from the Sample Characteristics Dictionary
    sample_char_dict = {0: ['patient identifier: 1', 'patient identifier: 3', 'patient identifier: 5', 'patient identifier: 7', 'patient identifier: 9', 'patient identifier: 10', 'patient identifier: 11', 'patient identifier: 13', 'patient identifier: 15', 'patient identifier: 24', 'patient identifier: 26', 'patient identifier: 28', 'patient identifier: 29', 'patient identifier: 30', 'patient identifier: 32', 'patient identifier: 33'], 1: ['disease: clear cell renal cell carcinoma (RCC)'], 2: ['tissue: kidney tumor'], 3: ['gender: male', 'gender: female'], 4: ['age at surgery (yrs): 51', 'age at surgery (yrs): 78', 'age at surgery (yrs): 53', 'age at surgery (yrs): 41', 'age at surgery (yrs): 39', 'age at surgery (yrs): 34', 'age at surgery (yrs): 66', 'age at surgery (yrs): 75', 'age at surgery (yrs): 40', 'age at surgery (yrs): 63', 'age at surgery (yrs): 35'], 5: ['fuhrman grade: III', 'fuhrman grade: IV', 'fuhrman grade: II'], 6: ['tumor size (cm): 18', 'tumor size (cm): 6', 'tumor size (cm): 8', 'tumor size (cm): 11', 'tumor size (cm): 6.5', 'tumor size (cm): 7', 'tumor size (cm): 5', 'tumor size (cm): 10', 'tumor size (cm): 15', 'tumor size (cm): 20', 'tumor size (cm): 8.5', 'tumor size (cm): 13', 'tumor size (cm): 4'], 7: ['necrosis: yes', 'necrosis: no'], 8: ['capsule infiltration: yes', 'capsule infiltration: no'], 9: ['tnm classification (t): 3c', 'tnm classification (t): 2', 'tnm classification (t): 3a', 'tnm classification (t): 1b', 'tnm classification (t): 3', 'tnm classification (t): 3b', 'tnm classification (t): 1', 'tnm classification (t): 1a'], 10: ['tnm classification (n): no data available', 'tnm classification (n): 1', 'tnm classification (n): 0', 'tnm classification (n): 2'], 11: ['tnm classification (m): no data available', 'tnm classification (m): 1'], 12: ['organ metastasis at surgery: no data available', 'organ metastasis at surgery: endocava, bones', 'organ metastasis at surgery: liver', 'organ metastasis at surgery: lung', 'organ metastasis at surgery: peritoneum'], 13: ['organ metastasis after surgery: no data available', 'organ metastasis after surgery: liver, spleen', 'organ metastasis after surgery: bones', 'organ metastasis after surgery: brain, lung, bones'], 14: ['patient status: cancer-specific death', 'patient status: alive without cancer'], 15: ['follow-up (months): 0', 'follow-up (months): 21', 'follow-up (months): 6', 'follow-up (months): 66', 'follow-up (months): 60', 'follow-up (months): 8', 'follow-up (months): 16', 'follow-up (months): 62', 'follow-up (months): 54', 'follow-up (months): 56', 'follow-up (months): 17']}
    
    # Since we have the dictionary, we'll create a DataFrame that matches the expected format
    # where rows are the feature indexes and columns are samples
    
    # First, get all the relevant row indices
    relevant_rows = [trait_row]
    if age_row is not None:
        relevant_rows.append(age_row)
    if gender_row is not None:
        relevant_rows.append(gender_row)
    
    # Create a DataFrame with the selected rows
    clinical_data = pd.DataFrame({row: sample_char_dict[row] for row in relevant_rows})
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(selected_clinical_df)
    print("Preview of extracted clinical features:")
    print(preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical data
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
I'll provide a corrected version of the code with proper Python syntax:

```python
import pandas as pd
import os
import json
import numpy as np
from typing import Callable, Optional, Dict, Any

# First, explore the directory to understand what data files are available
print("Contents of cohort directory:")
cohort_files = os.listdir(in_cohort_dir)
print(cohort_files)

# Check for any clinical data files with various common extensions
clinical_file = None
for file in cohort_files:
    if "clinical" in file.lower() or "characteristics" in file.lower() or "sample" in file.lower():
        clinical_file = os.path.join(in_cohort_dir, file)
        break

# Load and analyze the clinical data if available
clinical_data = None
if clinical_file:
    print(f"Found clinical data file: {clinical_file}")
    # Try different loading methods based on file extension
    if clinical_file.endswith('.pkl'):
        clinical_data = pd.read_pickle(clinical_file)
    elif clinical_file.endswith('.csv'):
        clinical_data = pd.read_csv(clinical_file)
    elif clinical_file.endswith('.tsv'):
        clinical_data = pd.read_csv(clinical_file, sep='\t')
    else:
        # Try to load as CSV by default
        try:
            clinical_data = pd.read_csv(clinical_file)
        except:
            try:
                clinical_data = pd.read_csv(clinical_file, sep='\t')
            except:
                print(f"Could not load {clinical_file} as CSV or TSV")

# If no specific clinical data file was found, check for a matrix file
if clinical_data is None:
    matrix_file = None
    for file in cohort_files:
        if "matrix" in file.lower() or "series_matrix" in file.lower():
            matrix_file = os.path.join(in_cohort_dir, file)
            break
            
    if matrix_file:
        print(f"Found matrix file: {matrix_file}")
        try:
            # For series matrix files, manually extract sample characteristics
            with open(matrix_file, 'r') as f:
                lines = f.readlines()
            
            # Extract sample characteristics section
            char_lines = [line for line in lines if line.startswith("!Sample_characteristics_ch1")]
            if char_lines:
                # Process and create a DataFrame
                sample_data = []
                for line in char_lines:
                    parts = line.strip().split('\t')
                    if len(parts) > 1:
                        sample_data.append(parts[1:])  # Skip the first part which is the header
                
                if sample_data:
                    # Transpose the data to get samples as rows
                    sample_data = list(map(list, zip(*sample_data)))
                    clinical_data = pd.DataFrame(sample_data)
                    # Try to identify if the first row contains headers
                    if all(isinstance(x, str) and ':' in x for x in clinical_data.iloc[0]):
                        # Extract feature names from the first samples
                        feature_names = [x.split(':', 1)[0].strip() for x in clinical_data.iloc[0]]
                        clinical_data.columns = feature_names
                    else:
                        clinical_data.columns = [f"characteristic_{i}" for i in range(clinical_data.shape[1])]
        except Exception as e:
            print(f"Error loading matrix file: {e}")

# Analyze the available data
is_gene_available = True  # Assume gene data is available unless we find evidence to the contrary

# Display clinical data if available
if clinical_data is not None:
    print("Clinical data preview:")
    print(clinical_data.head())
    
    # Try to identify trait, age, and gender information
    trait_row = None
    age_row = None
    gender_row = None
    
    # Look for columns or rows that might contain the relevant information
    for col in clinical_data.columns:
        col_data = clinical_data[col].astype(str).str.lower()
        # Check for trait data (tumor/normal status)
        if any(term in ' '.join(col_data) for term in ['tumor', 'normal', 'papillary', 'carcinoma', 'cancer']):
            trait_col = col
            trait_row = clinical_data.columns.get_loc(col)
            print(f"Found likely trait data in column {col} (row {trait_row}):")
            print(clinical_data[col].unique())
        
        # Check for age data
        if any(term in col.lower() for term in ['age', 'years']):
            age_col = col
            age_row = clinical_data.columns.get_loc(col)
            print(f"Found likely age data in column {col} (row {age_row}):")
            print(clinical_data[col].unique())
        
        # Check for gender data
        if any(term in ' '.join(col_data) for term in ['gender', 'sex', 'male', 'female']):
            gender_col = col
            gender_row = clinical_data.columns.get_loc(col)
            print(f"Found likely gender data in column {col} (row {gender_row}):")
            print(clinical_data[col].unique())
else:
    print("No clinical data found")
    trait_row = None

# Define conversion functions based on available data
def convert_trait(value):
    if value is None or pd.isna(value):
        return None
    
    value = str(value).lower()
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert tissue type to binary classification (0 for normal, 1 for tumor)
    if any(term in value for term in ['normal', 'adjacent', 'non-tumor', 'non tumor', 'control']):
        return 0
    elif any(term in value for term in ['tumor', 'papillary', 'carcinoma', 'cancer']):
        return 1
    else:
        return None

def convert_age(value):
    if value is None or pd.isna(value):
        return None
    
    value = str(value).lower()
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Try to extract age as a number
    try:
        # Look for digits in the string
        import re
        numbers = re.findall(r'\d+', value)
        if numbers:
            return float(numbers[0])
        else:
            return None
    except:
        return None

def convert_gender(value):
    if value is None or pd.isna(value):
        return None
    
    value = str(value).lower()
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert gender to binary (0 for female, 1 for male)
    if any(term in value for term in ['female', 'f', 'woman']):
        return 0
    elif any(term in value for term in ['male', 'm', 'man']):
        return 1
    else:
        return None

# Check trait availability
is_trait_available = trait_row is not None

# Perform initial validation and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Extract clinical features if trait data is available
if is_trait_available and clinical_data is not None:
    # Extract and process clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the processed clinical data
    print("


### Step 4: Gene Data Extraction

In [ ]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Add diagnostic code to check file content and structure
print("Examining matrix file structure...")
with gzip.open(matrix_file, 'rt') as file:
    table_marker_found = False
    lines_read = 0
    for i, line in enumerate(file):
        lines_read += 1
        if '!series_matrix_table_begin' in line:
            table_marker_found = True
            print(f"Found table marker at line {i}")
            # Read a few lines after the marker to check data structure
            next_lines = [next(file, "").strip() for _ in range(5)]
            print("First few lines after marker:")
            for next_line in next_lines:
                print(next_line)
            break
        if i < 10:  # Print first few lines to see file structure
            print(f"Line {i}: {line.strip()}")
        if i > 100:  # Don't read the entire file
            break
    
    if not table_marker_found:
        print("Table marker '!series_matrix_table_begin' not found in first 100 lines")
    print(f"Total lines examined: {lines_read}")

# 2. Try extracting gene expression data from the matrix file again with better diagnostics
try:
    print("\nAttempting to extract gene data from matrix file...")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {str(e)}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")

# If data extraction failed, try an alternative approach using pandas directly
if not is_gene_available:
    print("\nTrying alternative approach to read gene expression data...")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            # Skip lines until we find the marker
            for line in file:
                if '!series_matrix_table_begin' in line:
                    break
            
            # Try to read the data directly with pandas
            gene_data = pd.read_csv(file, sep='\t', index_col=0)
            
            if not gene_data.empty:
                print(f"Successfully extracted gene data with alternative method: {gene_data.shape}")
                print("First 20 gene IDs:")
                print(gene_data.index[:20])
                is_gene_available = True
            else:
                print("Alternative extraction method also produced empty data")
    except Exception as e:
        print(f"Alternative extraction failed: {str(e)}")


### Step 5: Gene Identifier Review

In [ ]:
# Examining the gene identifiers from the extracted data

# The gene IDs appear to be numerical indices (1, 2, 3, 4, etc.) rather than 
# standard human gene symbols like BRCA1, TP53, etc.
# 
# From the study description, this appears to be data from a custom-designed cDNA 
# microarray (GPL3985) that is "enriched in gene fragments that map to intronic regions 
# of known human genes" and focused on non-coding RNA profiling.
#
# These numerical identifiers would need to be mapped to proper gene symbols or 
# identifiers for meaningful analysis, as they are not standard gene symbols.

requires_gene_mapping = True


### Step 6: Gene Annotation

In [ ]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


### Step 7: Gene Identifier Mapping

In [ ]:
# 1. Decide which columns to use for gene mapping
# From the gene annotation, we need to identify which columns contain gene identifiers and gene symbols
# The 'ID' column in the gene annotation appears to contain the same type of numerical identifiers 
# as the gene expression data's index ('1', '2', etc.)
# The 'GENE_SYMBOL' column contains the actual gene symbols we want to map to

# 2. Create a gene mapping dataframe
probe_col = 'ID'
gene_col = 'GENE_SYMBOL'

# Check if the columns exist in the gene annotation data
if probe_col in gene_annotation.columns and gene_col in gene_annotation.columns:
    # Get the mapping between gene identifiers and gene symbols
    mapping_data = get_gene_mapping(gene_annotation, probe_col, gene_col)
    print(f"Created gene mapping with {len(mapping_data)} entries")
    print("First few rows of mapping data:")
    print(mapping_data.head())

    # 3. Apply the mapping to convert probe-level data to gene expression data
    gene_data = apply_gene_mapping(gene_data, mapping_data)
    print(f"Generated gene expression data for {len(gene_data.index)} unique genes")
    print("First few gene symbols in the processed data:")
    print(gene_data.index[:10].tolist())
    
    # Save the processed gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"Gene expression data saved to {out_gene_data_file}")
else:
    print(f"Required columns {probe_col} or {gene_col} not found in gene annotation data")
    # If mapping fails, we need to set gene availability to false
    is_gene_available = False


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the gene expression data
print("\nNormalizing gene symbols...")
try:
    # Load gene data that was saved in step 6
    gene_data = pd.read_csv(out_gene_data_file, index_col=0)
    print(f"Loaded gene data with {gene_data.shape[0]} genes and {gene_data.shape[1]} samples")
    
    # Normalize gene symbols using NCBI Gene database
    gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"After normalization: {gene_data.shape[0]} genes")
    
    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
    
    is_gene_available = True
except Exception as e:
    print(f"Error normalizing gene data: {e}")
    is_gene_available = False

# 2. Link clinical and genetic data
print("\nLinking clinical and genetic data...")
try:
    # Load clinical data from step 2
    clinical_data = pd.read_csv(out_clinical_data_file)
    print(f"Loaded clinical data with shape: {clinical_data.shape}")
    
    # Convert clinical data to proper format for linking (need to transpose for geo_link_clinical_genetic_data)
    clinical_transpose = clinical_data.set_index(trait).T
    
    # Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_transpose, gene_data)
    print(f"Created linked data with {linked_data.shape[0]} samples and {linked_data.shape[1]} features")
    is_trait_available = True
except Exception as e:
    print(f"Error linking clinical and genetic data: {e}")
    is_trait_available = False
    linked_data = pd.DataFrame()

# 3. Handle missing values in the linked data
if is_trait_available and not linked_data.empty:
    print("\nHandling missing values...")
    try:
        linked_data = handle_missing_values(linked_data, trait)
        print(f"After handling missing values: {linked_data.shape[0]} samples and {linked_data.shape[1]} features")
    except Exception as e:
        print(f"Error handling missing values: {e}")
        
    # 4. Determine whether the trait and demographic features are biased
    print("\nEvaluating feature bias...")
    try:
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        print(f"Trait bias determination: {is_biased}")
        print(f"Final linked data shape: {linked_data.shape[0]} samples and {linked_data.shape[1]} features")
    except Exception as e:
        print(f"Error evaluating feature bias: {e}")
        is_biased = True
else:
    print("\nSkipping missing value handling and bias evaluation as linked data is not available")
    is_biased = True

# 5. Validate and save cohort information
print("\nPerforming final validation...")
note = ""
if not is_trait_available:
    note = "Dataset does not contain required trait information"
elif is_biased:
    note = "Dataset has severe bias in the trait distribution"

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 6. Save the linked data if usable
print(f"\nDataset usability for {trait} association studies: {is_usable}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Final linked data saved to {out_data_file}")
else:
    if note:
        print(f"Reason: {note}")
    else:
        print("Dataset does not meet quality criteria for the specified trait")